In [ ]:
%matplotlib inline
%load_ext fortranmagic

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rc('figure', figsize=(12, 7))

# These will be used later.
jan2017 = pd.to_datetime(['2017-01-03 00:00:00+00:00',
 '2017-01-04 00:00:00+00:00',
 '2017-01-05 00:00:00+00:00',
 '2017-01-06 00:00:00+00:00',
 '2017-01-09 00:00:00+00:00',
 '2017-01-10 00:00:00+00:00',
 '2017-01-11 00:00:00+00:00',
 '2017-01-12 00:00:00+00:00',
 '2017-01-13 00:00:00+00:00',
 '2017-01-17 00:00:00+00:00'])
calendar = jan2017.values.astype('datetime64[D]')

event_dates = pd.to_datetime(['2017-01-06 00:00:00+00:00', 
                             '2017-01-07 00:00:00+00:00', 
                             '2017-01-08 00:00:00+00:00',
                             '2017-01-11 00:00:00+00:00']).values.astype('datetime64[D]')
event_values = np.array([10, 15, 20, 25])

def plot(xs, ys=None):
    if ys is None:
        return pd.Series(xs).plot()
    return pd.Series(index=xs, data=ys).plot()

ran_the_first_cell = True

<center>
  <h1>Foundations of Numerical Computing</h1>
  <h3>Scott Sanderson (GitHub: [@ssanderson](https://github.com/ssanderson), Twitter: [@scottbsanderson](https://twitter.com/scottbsanderson) )</h3>
  <h3>[https://github.com/ssanderson/foundations-of-numerical-computing](https://github.com/ssanderson/foundations-of-numerical-computing)</h3>
</center>

# About Me:

- Senior Engineer at [Quantopian](www.quantopian.com)
- I build tools for strangers on the internet to write quantitative trading strategies in Python.
- Most of those tools are built on top of numpy.

## Introduction

The goal of this tutorial is help you effectively solve numerically-intensive problems in Python.

In practice, doing numerical programming in Python means using [`numpy`](http://www.numpy.org/) (or a library built on top of `numpy`).

Using `numpy` effectively requires changing the way that you think about and solve problems.  TODO: Sentence here?

## Educational Philosophy

Programming problems require two kinds of knowledge:

- **Foundational Knowledge**
- **Domain Knowledge**

Foundational Knowledge can be applicable across a wide variety of domains

Domain Knowledge is more specific, hence more limited in applicability, but crucial when it applies!

I get a lot more bang for my educational buck by teaching foundational knowledge.

# Data Structures

> Rule 5. Data dominates. If you've chosen the right data structures and organized things well, the algorithms
will almost always be self-evident. Data structures, not algorithms, are central to programming.

- *Notes on Programming in C*, by Rob Pike.

# Review: Python Lists

In [ ]:
assert ran_the_first_cell, "You need to run the first cell!"

In [ ]:
l = [1, 'two', 3.0, 4, 5j, "six"]
l

In [ ]:
# Lists can be indexed like C-style arrays.
first = l[0]
second = l[1]
print("first:", first)
print("second:", second)

In [ ]:
# Negative indexing gives elements relative to the end of the list.
last = l[-1]
penultimate = l[-2]
print("last:", last)
print("second to last:", penultimate)

In [ ]:
# Lists can also be sliced, which makes a copy of elements between 
# start (inclusive) and stop (exclusive)
sublist = l[1:3]
sublist

In [ ]:
# l[:N] is equivalent to l[0:N].
first_three = l[:3]
first_three

In [ ]:
# l[3:] is equivalent to l[3:len(l)].
after_three = l[3:]
after_three

In [ ]:
# There's also a third parameter, "step", which gets every Nth element.
l = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h']
l[1:7:2]

In [ ]:
# Indexing with a step of -1 is a cute way to reverse a list.
l[::-1]

In [ ]:
# Lists can be grown efficiently (in O(1) amortized time).
l = [1, 2, 3, 4, 5]
print("Before:", l)
l.append('six')
print("After:", l)

In [ ]:
# Comprehensions let us perform elementwise computations.
l = [1, 2, 3, 4, 5]
[x * 2 for x in l]

## Review of Review: Python Lists

- Zero-indexed sequence of values.
- Can hold values of different types in a single list.
- Slicing syntax: `l[start:stop:step]` copies elements at regular intervals from `start` to `stop`.
- Efficient (`O(1)`) appends and removes from end.
- Comprehension syntax: `[f(x) for x in l if cond(x)]`.

<center><img src="images/pacino.gif" alt="Drawing" style="width: 100%;"/></center>

## Numerical Programming in Pure Python

In [ ]:
# Suppose we have some matrices...
a = [[1, 2, 3],
     [2, 3, 4],
     [5, 6, 7],
     [1, 1, 1]]

b = [[1, 2, 3, 4],
     [2, 3, 4, 5]]

In [ ]:
def matmul(A, B):
    """Multiply matrix A by matrix B."""
    
    rows_out = len(A)
    cols_out = len(B[0])
    out = [[0 for col in range(cols_out)] for row in range(rows_out)]
    
    for i in range(rows_out):
        for j in range(cols_out):
            for k in range(len(B)):
                out[i][j] += A[i][k] * B[k][j]
    return out

matmul(a, b)

<center><img src="images/gross.gif" alt="Drawing" style="width: 50%;"/></center>


In [ ]:
%%time
matmul(a, b)

In [ ]:
import random
def random_matrix(m, n):
    out = []
    for row in range(m):
        out.append([random.random() for _ in range(n)])
    return out

randm = random_matrix(2, 3)
randm

In [ ]:
%%time
randa = random_matrix(500, 100)
randb = random_matrix(100, 500)
x = matmul(randa, randb)

In [ ]:
# Maybe that's as good as we can do?  Let's try a simpler case.
def python_dot_product(xs, ys):
    return sum(x * y for x, y in zip(xs, ys))

In [ ]:
%%fortran
subroutine fortran_dot_product(xs, ys, result)
    double precision, intent(in) :: xs(:)
    double precision, intent(in) :: ys(:)
    double precision, intent(out) :: result
    
    result = sum(xs * ys)
end

In [ ]:
list_data = [float(i) for i in range(100000)]
array_data = np.array(list_data)

In [ ]:
%%time
python_dot_product(list_data, list_data)

In [ ]:
%%time
fortran_dot_product(array_data, array_data)

<center><img src="images/sloth.gif" alt="Drawing" style="width: 1080px;"/></center>

## Why is the Python Version so Much Slower?

In [ ]:
# Dynamic typing.
def multiply_elementwise(xs, ys):
    return [x * y for x, y in zip(xs, ys)]

multiply_elementwise([1, 2, 3, 4], [1, 2 + 0j, 3.0, 'four'])
#[type(x) for x in _]

In [ ]:
# Interpretation overhead.
source_code = 'a + b * c'
bytecode = compile(source_code, '', 'eval')
import dis; dis.dis(bytecode)

## Review: Why is the Python Version so Slow?
- Dynamic typing means that every single operation requires dispatching on the input type.
- Having an interpreter means that every instruction is fetched and dispatched at runtime.
- Other overheads:
  - Arbitrary-size integers.
  - Reference-counted garbage collection.

> This is the paradox that we have to work with when we're doing scientific or numerically-intensive Python. What makes Python fast for development -- this high-level, interpreted, and dynamically-typed aspect of the language -- is exactly what makes it slow for code execution.

- Jake VanderPlas, [*Losing Your Loops: Fast Numerical Computing with NumPy*](https://www.youtube.com/watch?v=EEUXKG97YRw)

# What Do We Do?

<center><img src="images/runaway.gif" alt="Drawing" style="width: 50%;"/></center>

<center><img src="images/thisisfine.gif" alt="Drawing" style="width: 1080px;"/></center>

- Normal Python is slow for numerical computation because it performs dynamic dispatch on every operation we perform...

- ...but often, we just want to do the same thing over and over in a loop!

- If we don't need Python's dynamicism, can we find a way to not pay (much) for it?

- **Idea:** Dispatch **once per logical operation** instead of **once per element**.

# Numpy

In [ ]:
import numpy as np

data = np.array([1, 2, 3, 4, 5, 6, 7, 8])

print("Data:", data)
print("===========")
print("DType:", data.dtype)
print("Shape:", data.shape)

In [ ]:
# Numpy provides operators that "vectorize" over the entire array.
data + data

In [ ]:
%%time
# Naive dot product
(array_data * array_data).sum()

In [ ]:
%%time
# Built-in dot product.
array_data.dot(array_data)

In [ ]:
%%time
fortran_dot_product(array_data, array_data)

In [ ]:
# Numpy won't allow us to write a string into an int array.
data[0] = "foo"

In [ ]:
# We also can't grow an array once it's created.
data.append(3)

In [ ]:
# We **can** reshape an array as long as the number of elements doesn't change.
reshaped = data.reshape(4, 2)
reshaped

Numpy arrays are:

- Fixed-type

- Size-immutable

- Multi-dimensional

- Fast\*

\* If you use them correctly.

# Exercises - Finding Help and Measuring Performance

## Solving Problems with Numpy - Overview

Numpy can be fast because it allows us to specify operations to perform against an entire array.

We can break down the tools provided by numpy into a few categories:

1. Universal Functions (UFuncs)
2. Selection
3. Reductions
4. Broadcasting

Taken together, these operations form an expressive toolbox that can solve a diverse set of problems.

## Creating Arrays

Numpy provides many functions for creating arrays.

We'll be using the following functions throughout the tutorial:

- `np.array`: Construct an array from a list of Python objects.
- `np.arange`: Numpy equivalent of the `range` function.
- `np.linspace`: Create an array from evenly-spaced points between two values.
- `np.full`/`np.ones`/`np.zeros`: Create an array of a given shape with a constant value.
- `np.full_like`/`np.ones_like`/`np.zeros_like`: Same, but using the shape of another array.

## Creating Arrays (cont'd)

- `np.eye`/`np.identity`/`np.diag`: Arrays with values on the diagonal. Useful for linear algebra.
- `np.random`: Randomly-generated values from various distributions.
- `np.load`/`np.save`: Load and save values from disk.
- `pandas.read_csv`: Read values from a CSV. Returns a [`pandas.DataFrame`](https://pandas.pydata.org/).

## Exercise  - Creating and Reshaping Arrays

## Solving Problems with Numpy - Universal Functions

Universal functions (ufuncs) are functions that can be applied across one or more arrays of the same shape.

They come in two main varieties:

- **Unary UFuncs** apply a 1-argument function to every element of an array.
- **Binary UFuncs** apply a 2-argument function to corresponding elements from two arrays.

In [ ]:
data = np.arange(15).reshape(3, 5)
data

In [ ]:
# Unary functions.
np.sqrt(data)

In [ ]:
# Binary operators.
data * data

In [ ]:
# Comparison operations
(data % 3) == 0

In [ ]:
# Boolean combinators.
((data % 2) == 0) & ((data % 3) == 0)

In [ ]:
# as of python 3.5, @ is matrix-multiply
data @ data.T

## UFunc Functions

All binary operator ufuncs have a normal function version:

- `+`,`-`,`*`,`/` are the same as `add`, `subtract`, `multiply`, `divide`
- `&`, `|`, `^`, `~` are the same as `bitwise_and`, `bitwise_or`, `bitwise_xor`, `bitwise_not`

Binary ufuncs are equipped with methods that implement common algorithms in terms of the ufunc.

- `reduce` computes a summary by inserting the binary operator between each element.
- `accumulate` is like `reduce`, but it outputs the intermediate values.
- `outer` computes the binary operator on every pair of elements from two arrays.

In [ ]:
data = (np.random.RandomState(1).randint(-10, 10, 18).reshape(3, 6))
data

In [ ]:
np.add.accumulate(data, axis=0)

# UFuncs Review

- UFuncs provide efficient elementwise operations applied across one or more arrays.
- Arithmetic Operators (`+`, `*`, `/`)
- Comparisons (`==`, `>`, `!=`)
- Boolean Operators (`&`, `|`, `^`)
- Trigonometric Functions (`sin`, `cos`)
- Transcendental Functions (`exp`, `log`)

# UFuncs Review (cont'd)

- UFunc Methods
  - `reduce`
  - `accumulate`
  - `outer`
  - `at` and `reduceat` (less common)

# UFunc Exercises

# Solving Problems with Numpy: Selections

When working with numpy, we often find that we need to operate on only a subset of our data.

Numpy arrays support the same set of rich slicing operations as Python lists, and they extend those semantics to support:

1. Selection of multiple indices.
2. Selection by boolean predicates.
3. Multiple dimensions.

In [ ]:
sines = np.sin(np.linspace(0, 3.14, 10))
cosines = np.cos(np.linspace(0, 3.14, 10))
sines

In [ ]:
# Indexing works with the same semantics as Python lists.
sines[0]

In [ ]:
sines[:3]  # First three elements  

In [ ]:
sines[5:]  # Elements from 5 on.

In [ ]:
sines[::2]  # Every other element.

In [ ]:
# More interesting: we can index with boolean arrays to filter by a predicate.
print("sines:\n", sines)
print("\nsines > 0.5:\n", sines > 0.5)
print("\nsines[sines > 0.5]:\n", sines[sines > 0.5])

In [ ]:
# We index with lists/arrays of integers to select values at those indices.
print(sines)
sines[[0, 4, 7]]

In [ ]:
# Index arrays are often used for sorting one or more arrays.
unsorted_data = np.array([1, 3, 2, 12, -1, 5, 2])

In [ ]:
sort_indices = np.argsort(unsorted_data)
sort_indices

In [ ]:
unsorted_data[sort_indices]

In [ ]:
prices = np.array([12, 6, 10, 5, 6])
tickers = np.array(['A', 'B', 'C', 'D', 'E'])

In [ ]:
# Sort assets by price by using the permutation that would sort market caps on ``assets``.
sorter = np.argsort(prices)
tickers[sorter]

In [ ]:
# Indexers are also useful for aligning data.
print("Values:\n", repr(event_values))
print("\nLabels:\n", repr(event_dates))
print("\nCalendar:\n", repr(calendar))

In [ ]:
print("Raw Dates:", event_dates)
print("Indices:", calendar.searchsorted(event_dates))
#print("Forward-Filled Dates:", calendar[calendar.searchsorted(event_dates)])

On multi-dimensional arrays, we can slice along each axis independently.

In [ ]:
data = np.arange(25).reshape(5, 5)
data

In [ ]:
data[1]             # Get the second row.
#data[:2, :2]       # Get the first two rows and first two columns.
#data[:2, [0, -1]]  # Get the first two rows, first/last columns.

In [ ]:
data[(data[:, 0] % 2) == 0]  # Rows where the first column is divisible by two.

Another common way to "select" data is to use `np.where` as a vectorized if-statement.

`np.where` takes three arguments: a boolean mask, an array of values to use for True locations, and an array of values to use for False locations.

In [ ]:
data = np.linspace(-5, 5, 9).reshape(3, 3)
data

In [ ]:
# We can implement a simple abs() function by selecting the data when the data > 0, and -data otherwise.
np.where(data > 0, data, -data)

# Selections Review

- Basic slicing operations work on Numpy arrays the same way they do on lists.
- Multidimensional arrays can apply selections independently along different axes.
- Indexing with a scalar removes a dimension.
- Indexing with slices, boolean arrays, index lists keep dimension unchanged.
  - Slice selects across axis with same rules as Python list.
  - Boolean array filters to coordinates where indexer is True.
  - Index list selects each corresponding index.
- `np.where` is like a vectorized if-statement.

# Selections Exercises

## Reductions

Another common class of operations in numpy is to "summarize" our data along a particular axis.

Examples of common "summaries" are:

- Minimum/Maximum
- Location of Minimum/Maximum (`argmin`/`argmax`)
- Sum/Product
- Statistical Moments (mean, variance, skew, kurtosis)

When we have a multidimensional array, we 

$Var(X) = \frac{1}{N}\sum_{i=1}^N (x_i - \bar{x})^2$

In [ ]:
def variance(x):
    return ((x - x.mean()) ** 2).sum() / len(x)

In [ ]:
xs = np.random.standard_normal(1000)
variance(xs)

In [ ]:
np.var(xs)

- ...we can do more interesting things with multi-dimensional arrays.

In [ ]:
data = np.arange(30).reshape(3, 10)
data

In [ ]:
data.mean()

In [ ]:
data.mean(axis=0)

In [ ]:
data.mean(axis=1)

## Reductions Review

- Reductions allow us to perform efficient aggregations over arrays.
- We can do aggregations over an entire array, or along a given axis.
- There are many built-in reductions:
  - `np.mean`
  - `np.var`
  - `np.median`
  - `np.count_nonzero`
  - `np.min`
  - `np.max`
  - ... many more in packages like `scipy`.

# Reductions Exercises

# Broadcasting

When working with multi-dimensional data, we often encounter situations where we have two "compatible" arrays of different dimensions, and we want to perform an operation on both arrays. NumPy's [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) system defines a set of rules that make this possible in a natural and intuitive way.

## Array Scalar Broadcasting

We've already seen one example of broadcasting in action: combining arrays with scalars.

Whenever you call a function that expects two arrays of the same shape, you can replace one of the arrays with a scalar.

In [ ]:
data = np.eye(3)
print(data)
#print(data * 2)

## Broadcasting with "Smaller" Arrays

Another common pattern where broadcasting is useful occurs when you have an N-dimensional array, and a "smaller" array whose dimensions are a subset of the larger array. 

A common way to create such an array is to apply one of the reductions we saw earlier.

In [ ]:
rng = np.random.RandomState(42)
data = rng.randint(-1, 5, (3, 6))
data

In [ ]:
# Subtract the mean of each column the values in the column. 
# This is a common operation when fitting statistical models.
print("Data:\n", data, sep='')
print("\nMean:\n", data.mean(axis=1), sep='')
print("\nData - Mean:\n", data - data.mean(axis=0), sep='')

In [ ]:
row = np.array([1, 2, 3, 4])
column = np.array([[1], [2], [3]])
print("Row:\n", row, sep='')
print("Column:\n", column, sep='')

In [ ]:
row + column

# Broadcasting Rules

Given arrays `x` and `y`:

1. If either array has a smaller dimension, make the dimensions match by padding 1s to the shape of the smaller array.
2. Iterate pairwise over the shapes of both arrays. At each coordinate:
   - If the sizes of the arrays match, do nothing.
   - If exactly one of the arrays has a dimension of size 1, make the dimensions match by "tiling" the smaller array along the current dimension.
   - If the arrays have unequal dimensions and neither is equal to 1, raise an error.

## Broadcasting Visualized

<center><img src="images/broadcasting.png" alt="Drawing" style="width: 60%;"/></center>

<h5>Source: http://www.scipy-lectures.org/_images/numpy_broadcasting.png</h5>

In [ ]:
x = np.linspace(-5, 5)
y = x[:, np.newaxis]  # Indexing with newaxis appends a dimension of length-1 to an array.

plt.imshow(np.sin(x) + np.cos(y));

# Broadcasting Review

- Numpy operations can work on arrays of different dimensions as long as the arrays' shapes are still "compatible".
- Broadcasting works by "tiling" the smaller array along the missing dimension.
- The result of a broadcasted operation is always at least as large in each dimension as the largest array in that dimension.

# Broadcasting Exercises

# Peeking Under the Hood

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

data = np.array([1, 2, 3, 4, 5, 6, 7, 8])
data

print("===========")
print("DType:", data.dtype)
print("Shape:", data.shape)
print("Strides:", data.strides)
print("Data:", data.data.tobytes())

# Example: Implementing Convolution with Strided Memory

# Review

- Numerical algorithms are slow in pure Python because the overhead of interpretation and dynamic dispatch dominates our runtime.

- Numpy solves this problem by:
  1. Imposing additional restrictions on the contents of arrays.
  2. Moving the inner loops of our algorithms into compiled C code.

- Using Numpy effectively requires reworking an algorithms to use vectorized operations instead of for-loops, but the resulting operations are often simpler, clearer, and faster than the pure Python equivalent.

## Review (cont'd)

**UFuncs**, **Selections**, **Reductions** are your core toolbox for construction algorithms with NumPy. 

**Broadcasting** allows you to naturally generalize algorithms over multiple dimensions. 

**Strided Memory** is an efficient and flexible underlying representation for arrays, but it can cause confusion when arrays are updated in place.